CER023 - Create Controller certificate
======================================

This notebook creates a certificate for the Controller endpoint. It
creates a controller-privatekey.pem as the private key and
controller-signingrequest.csr as the signing request.

The private key is a secret. The signing request (CSR) will be used by
the CA to generate a signed certificate for the service.

Steps
-----

### Parameters

In [ ]:
import getpass

app_name = "controller"
scaledset_name = "control"
container_name = "controller"
prefix_keyfile_name = "controller"
common_name = "controller-svc"

country_name = "US"
state_or_province_name = "Illinois"
locality_name = "Chicago"
organization_name = "Contoso"
organizational_unit_name = "Finance"
email_address = f"{getpass.getuser()}@contoso.com"

ssl_configuration_file = "service.openssl.cnf"

days = "825" # the number of days to certify the certificate for

test_cert_store_root = "/var/opt/secrets/test-certificates"

extendedKeyUsage = "extendedKeyUsage = critical, clientAuth, serverAuth"

### Common functions

Define helper functions used in this notebook.

In [ ]:
# Define `run` function for transient fault handling, suggestions on error, and scrolling updates on Windows
import sys
import os
import re
import json
import platform
import shlex
import shutil
import datetime

from subprocess import Popen, PIPE
from IPython.display import Markdown

retry_hints = {} # Output in stderr known to be transient, therefore automatically retry
error_hints = {} # Output in stderr where a known SOP/TSG exists which will be HINTed for further help
install_hint = {} # The SOP to help install the executable if it cannot be found

first_run = True
rules = None
debug_logging = False

def run(cmd, return_output=False, no_output=False, retry_count=0):
    """Run shell command, stream stdout, print stderr and optionally return output

    NOTES:

    1.  Commands that need this kind of ' quoting on Windows e.g.:

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='data-pool')].metadata.name}

        Need to actually pass in as '"':

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='"'data-pool'"')].metadata.name}

        The ' quote approach, although correct when pasting into Windows cmd, will hang at the line:
        
            `iter(p.stdout.readline, b'')`

        The shlex.split call does the right thing for each platform, just use the '"' pattern for a '
    """
    MAX_RETRIES = 5
    output = ""
    retry = False

    global first_run
    global rules

    if first_run:
        first_run = False
        rules = load_rules()

    # When running `azdata sql query` on Windows, replace any \n in """ strings, with " ", otherwise we see:
    #
    #    ('HY090', '[HY090] [Microsoft][ODBC Driver Manager] Invalid string or buffer length (0) (SQLExecDirectW)')
    #
    if platform.system() == "Windows" and cmd.startswith("azdata sql query"):
        cmd = cmd.replace("\n", " ")

    # shlex.split is required on bash and for Windows paths with spaces
    #
    cmd_actual = shlex.split(cmd)

    # Store this (i.e. kubectl, python etc.) to support binary context aware error_hints and retries
    #
    user_provided_exe_name = cmd_actual[0].lower()

    # When running python, use the python in the ADS sandbox ({sys.executable})
    #
    if cmd.startswith("python "):
        cmd_actual[0] = cmd_actual[0].replace("python", sys.executable)

        # On Mac, when ADS is not launched from terminal, LC_ALL may not be set, which causes pip installs to fail
        # with:
        #
        #    UnicodeDecodeError: 'ascii' codec can't decode byte 0xc5 in position 4969: ordinal not in range(128)
        #
        # Setting it to a default value of "en_US.UTF-8" enables pip install to complete
        #
        if platform.system() == "Darwin" and "LC_ALL" not in os.environ:
            os.environ["LC_ALL"] = "en_US.UTF-8"

    # When running `kubectl`, if AZDATA_OPENSHIFT is set, use `oc`
    #
    if cmd.startswith("kubectl ") and "AZDATA_OPENSHIFT" in os.environ:
        cmd_actual[0] = cmd_actual[0].replace("kubectl", "oc")

    # To aid supportabilty, determine which binary file will actually be executed on the machine
    #
    which_binary = None

    # Special case for CURL on Windows.  The version of CURL in Windows System32 does not work to
    # get JWT tokens, it returns "(56) Failure when receiving data from the peer".  If another instance
    # of CURL exists on the machine use that one.  (Unfortunately the curl.exe in System32 is almost
    # always the first curl.exe in the path, and it can't be uninstalled from System32, so here we
    # look for the 2nd installation of CURL in the path)
    if platform.system() == "Windows" and cmd.startswith("curl "):
        path = os.getenv('PATH')
        for p in path.split(os.path.pathsep):
            p = os.path.join(p, "curl.exe")
            if os.path.exists(p) and os.access(p, os.X_OK):
                if p.lower().find("system32") == -1:
                    cmd_actual[0] = p
                    which_binary = p
                    break

    # Find the path based location (shutil.which) of the executable that will be run (and display it to aid supportability), this
    # seems to be required for .msi installs of azdata.cmd/az.cmd.  (otherwise Popen returns FileNotFound) 
    #
    # NOTE: Bash needs cmd to be the list of the space separated values hence shlex.split.
    #
    if which_binary == None:
        which_binary = shutil.which(cmd_actual[0])

    if which_binary == None:
        if user_provided_exe_name in install_hint and install_hint[user_provided_exe_name] is not None:
            display(Markdown(f'HINT: Use [{install_hint[user_provided_exe_name][0]}]({install_hint[user_provided_exe_name][1]}) to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)")
    else:   
        cmd_actual[0] = which_binary

    start_time = datetime.datetime.now().replace(microsecond=0)

    print(f"START: {cmd} @ {start_time} ({datetime.datetime.utcnow().replace(microsecond=0)} UTC)")
    print(f"       using: {which_binary} ({platform.system()} {platform.release()} on {platform.machine()})")
    print(f"       cwd: {os.getcwd()}")

    # Command-line tools such as CURL and AZDATA HDFS commands output
    # scrolling progress bars, which causes Jupyter to hang forever, to
    # workaround this, use no_output=True
    #

    # Work around a infinite hang when a notebook generates a non-zero return code, break out, and do not wait
    #
    wait = True 

    try:
        if no_output:
            p = Popen(cmd_actual)
        else:
            p = Popen(cmd_actual, stdout=PIPE, stderr=PIPE, bufsize=1)
            with p.stdout:
                for line in iter(p.stdout.readline, b''):
                    line = line.decode()
                    if return_output:
                        output = output + line
                    else:
                        if cmd.startswith("azdata notebook run"): # Hyperlink the .ipynb file
                            regex = re.compile('  "(.*)"\: "(.*)"') 
                            match = regex.match(line)
                            if match:
                                if match.group(1).find("HTML") != -1:
                                    display(Markdown(f' - "{match.group(1)}": "{match.group(2)}"'))
                                else:
                                    display(Markdown(f' - "{match.group(1)}": "[{match.group(2)}]({match.group(2)})"'))

                                    wait = False
                                    break # otherwise infinite hang, have not worked out why yet.
                        else:
                            print(line, end='')
                            if rules is not None:
                                apply_expert_rules(line)

        if wait:
            p.wait()
    except FileNotFoundError as e:
        if install_hint is not None:
            display(Markdown(f'HINT: Use {install_hint} to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)") from e

    exit_code_workaround = 0 # WORKAROUND: azdata hangs on exception from notebook on p.wait()

    if not no_output:
        for line in iter(p.stderr.readline, b''):
            try:
                line_decoded = line.decode()
            except UnicodeDecodeError:
                # NOTE: Sometimes we get characters back that cannot be decoded(), e.g.
                #
                #   \xa0
                #
                # For example see this in the response from `az group create`:
                #
                # ERROR: Get Token request returned http error: 400 and server 
                # response: {"error":"invalid_grant",# "error_description":"AADSTS700082: 
                # The refresh token has expired due to inactivity.\xa0The token was 
                # issued on 2018-10-25T23:35:11.9832872Z
                #
                # which generates the exception:
                #
                # UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 179: invalid start byte
                #
                print("WARNING: Unable to decode stderr line, printing raw bytes:")
                print(line)
                line_decoded = ""
                pass
            else:

                # azdata emits a single empty line to stderr when doing an hdfs cp, don't
                # print this empty "ERR:" as it confuses.
                #
                if line_decoded == "":
                    continue
                
                print(f"STDERR: {line_decoded}", end='')

                if line_decoded.startswith("An exception has occurred") or line_decoded.startswith("ERROR: An error occurred while executing the following cell"):
                    exit_code_workaround = 1

                # inject HINTs to next TSG/SOP based on output in stderr
                #
                if user_provided_exe_name in error_hints:
                    for error_hint in error_hints[user_provided_exe_name]:
                        if line_decoded.find(error_hint[0]) != -1:
                            display(Markdown(f'HINT: Use [{error_hint[1]}]({error_hint[2]}) to resolve this issue.'))

                # apply expert rules (to run follow-on notebooks), based on output
                #
                if rules is not None:
                    apply_expert_rules(line_decoded)

                # Verify if a transient error, if so automatically retry (recursive)
                #
                if user_provided_exe_name in retry_hints:
                    for retry_hint in retry_hints[user_provided_exe_name]:
                        if line_decoded.find(retry_hint) != -1:
                            if retry_count < MAX_RETRIES:
                                print(f"RETRY: {retry_count} (due to: {retry_hint})")
                                retry_count = retry_count + 1
                                output = run(cmd, return_output=return_output, retry_count=retry_count)

                                if return_output:
                                    return output
                                else:
                                    return

    elapsed = datetime.datetime.now().replace(microsecond=0) - start_time

    # WORKAROUND: We avoid infinite hang above in the `azdata notebook run` failure case, by inferring success (from stdout output), so
    # don't wait here, if success known above
    #
    if wait: 
        if p.returncode != 0:
            raise SystemExit(f'Shell command:\n\n\t{cmd} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(p.returncode)}.\n')
    else:
        if exit_code_workaround !=0 :
            raise SystemExit(f'Shell command:\n\n\t{cmd} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(exit_code_workaround)}.\n')

    print(f'\nSUCCESS: {elapsed}s elapsed.\n')

    if return_output:
        return output

def load_json(filename):
    """Load a json file from disk and return the contents"""

    with open(filename, encoding="utf8") as json_file:
        return json.load(json_file)

def load_rules():
    """Load any 'expert rules' from the metadata of this notebook (.ipynb) that should be applied to the stderr of the running executable"""

    try:

        # Load this notebook as json to get access to the expert rules in the notebook metadata.
        #
        j = load_json("cer023-create-controller-cert.ipynb")

    except:
        pass # If the user has renamed the book, we can't load ourself.  NOTE: Is there a way in Jupyter, to know your own filename?

    else:
        if "metadata" in j and \
            "azdata" in j["metadata"] and \
            "expert" in j["metadata"]["azdata"] and \
            "rules" in j["metadata"]["azdata"]["expert"]:

            rules = j["metadata"]["azdata"]["expert"]["rules"]

            rules.sort() # Sort rules, so they run in priority order (the [0] element).  Lowest value first.

            # print (f"EXPERT: There are {len(rules)} rules to evaluate.")

            return rules

def apply_expert_rules(line):
    """Determine if the stderr line passed in, matches the regular expressions for any of the 'expert rules', if so
    inject a 'HINT' to the follow-on SOP/TSG to run"""

    global rules

    for rule in rules:

        # rules that have 9 elements are the injected (output) rules (the ones we want).  Rules
        # with only 8 elements are the source (input) rules, which are not expanded (i.e. TSG029,
        # not ../repair/tsg029-nb-name.ipynb)
        if len(rule) == 9:
            notebook = rule[1]
            cell_type = rule[2]
            output_type = rule[3] # i.e. stream or error
            output_type_name = rule[4] # i.e. ename or name 
            output_type_value = rule[5] # i.e. SystemExit or stdout
            details_name = rule[6]  # i.e. evalue or text 
            expression = rule[7].replace("\\*", "*") # Something escaped *, and put a \ in front of it!

            if debug_logging:
                print(f"EXPERT: If rule '{expression}' satisfied', run '{notebook}'.")

            if re.match(expression, line, re.DOTALL):

                if debug_logging:
                    print("EXPERT: MATCH: name = value: '{0}' = '{1}' matched expression '{2}', therefore HINT '{4}'".format(output_type_name, output_type_value, expression, notebook))

                match_found = True

                display(Markdown(f'HINT: Use [{notebook}]({notebook}) to resolve this issue.'))




print('Common functions defined successfully.')

# Hints for binary (transient fault) retry, (known) error and install guide
#
retry_hints = {'kubectl': ['A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'], 'azdata': ['Endpoint sql-server-master does not exist', 'Endpoint livy does not exist', 'Failed to get state for cluster', 'Endpoint webhdfs does not exist', 'Adaptive Server is unavailable or does not exist', 'Error: Address already in use']}
error_hints = {'kubectl': [['no such host', 'TSG010 - Get configuration contexts', '../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb'], ['no such host', 'TSG011 - Restart sparkhistory server', '../repair/tsg011-restart-sparkhistory-server.ipynb'], ['No connection could be made because the target machine actively refused it', 'TSG056 - Kubectl fails with No connection could be made because the target machine actively refused it', '../repair/tsg056-kubectl-no-connection-could-be-made.ipynb']], 'azdata': [['azdata login', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['The token is expired', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Reason: Unauthorized', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Max retries exceeded with url: /api/v1/bdc/endpoints', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Look at the controller logs for more details', 'TSG027 - Observe cluster deployment', '../diagnose/tsg027-observe-bdc-create.ipynb'], ['provided port is already allocated', 'TSG062 - Get tail of all previous container logs for pods in BDC namespace', '../log-files/tsg062-tail-bdc-previous-container-logs.ipynb'], ['Create cluster failed since the existing namespace', 'SOP061 - Delete a big data cluster', '../install/sop061-delete-bdc.ipynb'], ['Failed to complete kube config setup', 'TSG067 - Failed to complete kube config setup', '../repair/tsg067-failed-to-complete-kube-config-setup.ipynb'], ['Error processing command: "ApiError', 'TSG110 - Azdata returns ApiError', '../repair/tsg110-azdata-returns-apierror.ipynb'], ['Error processing command: "ControllerError', 'TSG036 - Controller logs', '../log-analyzers/tsg036-get-controller-logs.ipynb'], ['ERROR: 500', 'TSG046 - Knox gateway logs', '../log-analyzers/tsg046-get-knox-logs.ipynb'], ['Data source name not found and no default driver specified', 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ["Can't open lib 'ODBC Driver 17 for SQL Server", 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ['Control plane upgrade failed. Failed to upgrade controller.', 'TSG108 - View the controller upgrade config map', '../diagnose/tsg108-controller-failed-to-upgrade.ipynb']]}
install_hint = {'kubectl': ['SOP036 - Install kubectl command line interface', '../install/sop036-install-kubectl.ipynb'], 'azdata': ['SOP063 - Install azdata CLI (using package manager)', '../install/sop063-packman-install-azdata.ipynb']}

### Get the Kubernetes namespace for the big data cluster

Get the namespace of the Big Data Cluster use the kubectl command line
interface .

**NOTE:**

If there is more than one Big Data Cluster in the target Kubernetes
cluster, then either:

-   set \[0\] to the correct value for the big data cluster.
-   set the environment variable AZDATA\_NAMESPACE, before starting
    Azure Data Studio.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

if "AZDATA_NAMESPACE" in os.environ:
    namespace = os.environ["AZDATA_NAMESPACE"]
else:
    try:
        namespace = run(f'kubectl get namespace --selector=MSSQL_CLUSTER -o jsonpath={{.items[0].metadata.name}}', return_output=True)
    except:
        from IPython.display import Markdown
        print(f"ERROR: Unable to find a Kubernetes namespace with label 'MSSQL_CLUSTER'.  SQL Server Big Data Cluster Kubernetes namespaces contain the label 'MSSQL_CLUSTER'.")
        display(Markdown(f'HINT: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
        raise

print(f'The SQL Server Big Data Cluster Kubernetes namespace is: {namespace}')

### Create a temporary directory to stage files

In [ ]:
# Create a temporary directory to hold configuration files

import tempfile

temp_dir = tempfile.mkdtemp()

print(f"Temporary directory created: {temp_dir}")

### Helper function to save configuration files to disk

In [ ]:
# Define helper function 'save_file' to save configuration files to the temporary directory created above
import os
import io

def save_file(filename, contents):
    with io.open(os.path.join(temp_dir, filename), "w", encoding='utf8', newline='\n') as text_file:
      text_file.write(contents)

      print("File saved: " + os.path.join(temp_dir, filename))

print("Function `save_file` defined successfully.")

### Get endpoint hostname

In [ ]:
import json
import urllib

endpoint = run(f'azdata bdc endpoint list --endpoint="{app_name}"', return_output=True)
endpoint = json.loads(endpoint)
endpoint = endpoint['endpoint']

print(f"endpoint: {endpoint}")

hostname = urllib.parse.urlparse(endpoint).hostname

print(f"hostname: {hostname}")

### Get name of the ‘Running’ `controller` `pod`

In [ ]:
# Place the name  of the 'Running' controller pod in variable `controller`

controller = run(f'kubectl get pod --selector=app=controller -n {namespace} -o jsonpath={{.items[0].metadata.name}} --field-selector=status.phase=Running', return_output=True)

print(f"Controller pod name: {controller}")

### Create the DNS alt\_names for data plane in secure clusters

Get the cluster configuration from the Big Data Cluster using
`azdata bdc config`, and pull the Active Directory DNS names out of it,
and place them into the certificate configuration file as DNS alt\_names

In [ ]:
import json

alt_names = ""

bdc_config = run("azdata bdc config show", return_output=True)
bdc_config = json.loads(bdc_config)

dns_counter = 3 # DNS.1 and DNS.2 are already in the certificate template.

# Add entry for "DNS.3 = {common_name}.<your_ad_domain_name>.local"
#
if "security" in bdc_config["spec"] and "activeDirectory" in bdc_config["spec"]["security"]:
    domain_dns_name = bdc_config["spec"]["security"]["activeDirectory"]["domainDnsName"]

    alt_names += f"DNS.{str(dns_counter)} = {common_name}.{domain_dns_name}\n"
    dns_counter = dns_counter + 1

# Add entry for "DNS.4 = <your_endpoint_name>.<your_ad_domain_name>.local"
#
if app_name in bdc_config["spec"]["resources"]:
    app_name_endpoints = bdc_config["spec"]["resources"][app_name]["spec"]["endpoints"]
    for endpoint in app_name_endpoints:
        if "dnsName" in endpoint:
            alt_names += f'DNS.{str(dns_counter)} = {endpoint["dnsName"]}\n'
            dns_counter = dns_counter + 1

# Special case for the controller certificate
#
if app_name == "controller":
    alt_names += f"DNS.{str(dns_counter)} = localhost\n"
    dns_counter = dns_counter + 1

print("DNS alt_names (data plane):")
print(alt_names)

### Create the DNS alt\_names for control plane in secure clusters

Get the cluster configuration from the Big Data Cluster using
`azdata bdc endpoint list`, and pull the Active Directory DNS names out
of it for the control plane expternal endpoints (Controller and
Management Proxy), and place them into the certificate configuration
file as DNS alt\_names

In [ ]:
import json
from urllib.parse import urlparse

if app_name == "controller" or app_name == "mgmtproxy":
    bdc_endpoint_list = run("azdata bdc endpoint list", return_output=True)
    bdc_endpoint_list = json.loads(bdc_endpoint_list)

    # Parse the DNS host name from:
    #
    #    "endpoint": "https://monitor.aris.local:30777"
    # 
    for endpoint in bdc_endpoint_list:
        if endpoint["name"] == app_name:
            url = urlparse(endpoint["endpoint"])
            alt_names += f"DNS.{str(dns_counter)} = {url.hostname}\n"
            dns_counter = dns_counter + 1

print("DNS alt_names (control plane):")
print(alt_names)

### Create alt\_names

If the Kuberenetes service is of “NodePort” type, then the IP address
needed to validate the cluster certificate could be for any node in the
Kubernetes cluster, so here all node IP addresses in the Big Data
Cluster are added as alt\_names. Otherwise (if not NodePort, and
therefore LoadBalancer), add just the hostname as returned from
`azdata bdc endpoint list` above.

In [ ]:
service_type = run(f"kubectl get svc {common_name}-external -n {namespace} -o jsonpath={{.spec.type}}", return_output=True)

print(f"Service type for '{common_name}-external' is: '{service_type}'")
print("")

if service_type == "NodePort":
    nodes_ip_address = run("kubectl ""get nodes -o jsonpath={.items[*].status.addresses[0].address}""", return_output=True)
    nodes_ip_address = nodes_ip_address.split(' ')

    counter = 1
    for ip in nodes_ip_address:
        alt_names += f"IP.{counter} = {ip}\n"
        counter = counter + 1
else:
    alt_names += f"IP.1 = {hostname}\n"

print("All (DNS and IP) alt_names:")
print(alt_names)

### Generate Certificate Configuration file

NOTE: There is a special case for the `controller` certificate, that
needs to be generated in PKCS\#1 format.

In [ ]:
certificate = f"""
[ req ]
# Options for the `req` tool (`man req`).
default_bits        = 2048
default_keyfile     = {test_cert_store_root}/{app_name}/{prefix_keyfile_name}-privatekey{".pkcs8" if app_name == "controller" else ""}.pem
distinguished_name  = req_distinguished_name
string_mask         = utf8only

# SHA-1 is deprecated, so use SHA-2 instead.
default_md          = sha256
req_extensions      = v3_req

[ req_distinguished_name ]
countryName         = Country Name (2 letter code)
countryName_default = {country_name}

stateOrProvinceName         = State or Province Name (full name)
stateOrProvinceName_default = {state_or_province_name}

localityName                = Locality Name (eg, city)
localityName_default        = {locality_name}

organizationName            = Organization Name (eg, company)
organizationName_default    = {organization_name}

organizationalUnitName         = Organizational Unit (eg, division)
organizationalUnitName_default = {organizational_unit_name}

commonName         = Common Name (e.g. server FQDN or YOUR name)
commonName_default = {common_name}

emailAddress         = Email Address
emailAddress_default = {email_address}

[ v3_req ]
subjectAltName       = @alt_names
subjectKeyIdentifier = hash
basicConstraints     = CA:FALSE
keyUsage             = digitalSignature, keyEncipherment
{extendedKeyUsage}

[ alt_names ]
DNS.1 = {common_name}
DNS.2 = {common_name}.{namespace}.svc.cluster.local  # Use the namespace applicable for your cluster
{alt_names}
"""

save_file(ssl_configuration_file, certificate)

### Copy certificate configuration to `controller` `pod`

In [ ]:
import os

cwd = os.getcwd()
os.chdir(temp_dir) # Use chdir to workaround kubectl bug on Windows, which incorrectly processes 'c:\' on kubectl cp cmd line 

run(f'kubectl exec {controller} -c controller -n {namespace} -- bash -c "mkdir -p {test_cert_store_root}/{app_name}"')

run(f'kubectl cp {ssl_configuration_file} {controller}:{test_cert_store_root}/{app_name}/{ssl_configuration_file} -c controller -n {namespace}')

os.chdir(cwd)

### Generate certificate

Use openssl req to generate a certificate in PKCS\#10 format. See:

-   https://www.openssl.org/docs/man1.0.2/man1/req.html

In [ ]:
cmd = f"openssl req -config {test_cert_store_root}/{app_name}/service.openssl.cnf -newkey rsa:2048 -sha256 -nodes -days {days} -out {test_cert_store_root}/{app_name}/{prefix_keyfile_name}-signingrequest.csr -outform PEM -subj '/C={country_name}/ST={state_or_province_name}/L={locality_name}/O={organization_name}/OU={organizational_unit_name}/CN={common_name}'"

run(f'kubectl exec {controller} -n {namespace} -c controller -- bash -c "{cmd}"')

### Convert the private key to PKCS12 format

The private key for controller needs to be converted to PKCS12 format.

In [ ]:
cmd = f'openssl rsa -in {test_cert_store_root}/{app_name}/{prefix_keyfile_name}-privatekey.pkcs8.pem -out {test_cert_store_root}/{app_name}/{prefix_keyfile_name}-privatekey.pem'

run(f'kubectl exec {controller} -n {namespace} -c controller -- bash -c "{cmd}"')

### Clean up temporary directory for staging configuration files

In [ ]:
# Delete the temporary directory used to hold configuration files

import shutil

shutil.rmtree(temp_dir)

print(f'Temporary directory deleted: {temp_dir}')

In [ ]:
print('Notebook execution complete.')

Related
-------

-   [CER030 - Sign Management Proxy certificate with generated
    CA](../cert-management/cer030-sign-service-proxy-generated-cert.ipynb)

-   [CER032 - Sign App-Proxy certificate with generated
    CA](../cert-management/cer032-sign-app-proxy-generated-cert.ipynb)

-   [CER042 - Install signed App-Proxy
    certificate](../cert-management/cer042-install-app-proxy-cert.ipynb)